### Image classification 
- [venv] anaconda/py310
- [date] 2024/12/23
- [posi] 

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.transforms.functional import crop
from torch.utils.data import TensorDataset
from torch.optim import Adam
from torchvision import transforms, datasets
from torchvision.models import efficientnet_b2, EfficientNet_B2_Weights


from skorch import NeuralNetClassifier
from skorch.callbacks import Checkpoint
from skorch.helper import predefined_split
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader, Subset

In [87]:
# 1. 데이터 로드 및 Augmentation 설정
data_dir = './glasses'  # 데이터셋 루트 경로
batch_size = 32

In [97]:
# 2. 사용자 정의 Transform: image crop
class CustomCrop:
    def __init__(self, top, left, height, width):
        self.top = top
        self.left = left
        self.height = height
        self.width = width

    def __call__(self, image):
        return crop(image, self.top, self.left, self.height, self.width)


crop_transform = CustomCrop(top=0, left=90, height=1050, width=1500)

In [98]:
# 데이터 증강 설정
train_transforms = transforms.Compose([
    crop_transform,
    transforms.Resize((300, 300)),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.RandomResizedCrop(260, scale=(0.87, 0.87)),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [112]:
valid_transforms = transforms.Compose([
    crop_transform,
    transforms.Resize((260, 260)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [113]:
# 전체 데이터 로드
dataset = datasets.ImageFolder(root=data_dir, transform=None)
dataset, dataset.classes

(Dataset ImageFolder
     Number of datapoints: 106
     Root location: ./glasses,
 ['break', 'shiny'])

In [132]:
# Train/Validation 분리
train_idx, valid_idx = train_test_split(
    np.arange(len(dataset)),
    test_size=0.1,  # 10%를 Validation 데이터로 사용
    stratify=dataset.targets,  # 클래스 비율을 유지
    shuffle=True,            
    random_state=42        
)

In [133]:
# Subset으로 Train/Valid 데이터 생성
train_subset = Subset(dataset, train_idx)
valid_subset = Subset(dataset, valid_idx)

# Subset에 각각 Transform 적용
train_subset.dataset.transform = train_transforms
valid_subset.dataset.transform = valid_transforms

In [134]:
image, label = train_subset[0]
len(train_subset), image.shape, label

(95, torch.Size([3, 260, 260]), 1)

In [135]:
X_train = np.array([image for image,_ in train_subset]) 
y_train = np.array([label for _,label in train_subset])
X_valid = np.array([image for image,_ in valid_subset]) 
y_valid = np.array([label for _,label in valid_subset])

In [136]:
# PyTorch TensorDataset으로 변환
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(X_valid), torch.tensor(y_valid, dtype=torch.long))

In [150]:
# 2. EfficientNet-B2 모델 정의 및 수정
class EfficientNetB2Model(nn.Module):
    def __init__(self, pretrained=True, num_classes=2):
        super().__init__()
        #self.base_model = models.efficientnet_b2(pretrained=pretrained)
        self.base_model = efficientnet_b2(weights=EfficientNet_B2_Weights.DEFAULT)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)  # 1408->2

    def forward(self, x):
        return self.base_model(x)
        #return F.softmax(o2, dim=1)


# 모델 인스턴스화
model = EfficientNetB2Model(pretrained=True, num_classes=2)

In [138]:
# 3. Skorch NeuralNetClassifier 정의
net = NeuralNetClassifier(
    model,
    criterion=nn.CrossEntropyLoss,
    optimizer=Adam,
    optimizer__lr=1e-3,
    max_epochs=1000,
    batch_size=batch_size,
    iterator_train__shuffle=True,
    iterator_valid__shuffle=False,
    train_split=predefined_split(valid_dataset),   
    callbacks=[Checkpoint(f_params='best_params.pt')],
    device='cuda' if torch.cuda.is_available() else 'cpu'   
)

In [139]:
net.fit(train_dataset, y=None)

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.5439       0.7273        0.5115     +  0.6806
      2        0.1252       0.7273        0.3271     +  0.5151
      3        0.0120       0.7273        0.3842        0.4838
      4        0.0046       0.7273        0.4273        0.5046
      5        0.0046       0.6364        0.5275        0.4847
      6        0.0029       0.7273        0.5772        0.4957
      7        0.0197       0.7273        0.3426        0.4412
      8        0.0011       0.8182        0.3470        0.4581
      9        0.0647       0.8182        0.6999        0.4527
     10        0.1153       0.9091        0.1248     +  0.4467
     11        0.0503       1.0000        0.0217     +  0.5283
     12        0.0187       1.0000        0.0081     +  0.4692
     13        0.0190       1.0000        0.0232        0.4529
     14        0.0176       1.0000        0.0133       

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=EfficientNetB2Model(
    (base_model): EfficientNet(
      (features): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): Sequential(
          (0): MBConv(
            (block): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): SiLU(inplace=True)
              )
              (1): SqueezeExcitation(
                (avgpool): AdaptiveAvgPool2d(output_size=1)
                (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
                (fc2

In [151]:
#torch.save(model.state_dict(), "best_params_0001.pt")
model.load_state_dict(torch.load("best_params_0001.pt", weights_only=True))

<All keys matched successfully>

In [154]:
res = net.predict(valid_dataset)
res, y_valid

(array([1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]),
 array([1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]))

In [155]:
# 5. 모델 평가
accuracy = net.score(X_valid, y=y_valid)
print(f'Validation Accuracy: {accuracy:.2f}')

Validation Accuracy: 1.00


In [128]:
res = net.predict(train_dataset)
res, y_train

(array([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 1]),
 array([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 1]))